In [1]:
import pandas as pd
# Charger le dataset
df = pd.read_csv('WA_Fn-UseC_-Marketing-Customer-Value-Analysis.csv')

In [2]:
# Aperçu des données
df.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Customer                       9134 non-null   object 
 1   State                          9134 non-null   object 
 2   Customer Lifetime Value        9134 non-null   float64
 3   Response                       9134 non-null   object 
 4   Coverage                       9134 non-null   object 
 5   Education                      9134 non-null   object 
 6   Effective To Date              9134 non-null   object 
 7   EmploymentStatus               9134 non-null   object 
 8   Gender                         9134 non-null   object 
 9   Income                         9134 non-null   int64  
 10  Location Code                  9134 non-null   object 
 11  Marital Status                 9134 non-null   object 
 12  Monthly Premium Auto           9134 non-null   i

In [4]:
# Calcul des agrégations par Sales Channel
calcul_agg = df.groupby("Sales Channel").agg(
    Total_Customers=("Customer", "count"),
    Sum_CLV=("Customer Lifetime Value", "sum"),
    Total_Premium=("Monthly Premium Auto", "sum"),
    Total_Claim_Amount=("Total Claim Amount", "sum")
).reset_index()

# Calcul des totaux globaux
total_row = pd.DataFrame({
    "Sales Channel": ["Total"],
    "Total_Customers": [df["Customer"].count()],
    "Sum_CLV": [df["Customer Lifetime Value"].sum()],
    "Total_Premium": [df["Monthly Premium Auto"].sum()],
    "Total_Claim_Amount": [df["Total Claim Amount"].sum()]
})

# Ajouter la ligne des totaux globaux au tableau final
calcul_agg = pd.concat([calcul_agg, total_row], ignore_index=True)

In [5]:
# Désactiver la notation scientifique
pd.set_option('display.float_format', '{:.2f}'.format)
# Afficher le tableau
display(calcul_agg)

,Sales Channel,Total_Customers,Sum_CLV,Total_Premium,Total_Claim_Amount
0,Agent,3477,27668955.42,325722,1524437.36
1,Branch,2567,20843300.35,239519,1111169.08
2,Call Center,1765,14296651.35,163231,755639.96
3,Web,1325,10308219.18,122993,573720.65
4,Total,9134,73117126.30,851465,3964967.05


In [6]:
# Définition des coûts marketing simulés
# Compte tenu que la somme minimum de premium pour un canal est environ 100000,
# on suppose un budget marketing de 100000 par canal
total_marketing_cost = 100000

# Calcul des KPIs par canal de vente
kpi_results = df.groupby("Sales Channel").agg(
    Total_Customers=("Customer", "count"),
    Avg_CLV=("Customer Lifetime Value", "mean"),
    Total_Premium=("Monthly Premium Auto", "sum"),
    Conversion_Rate=("Response", lambda x: (x == "Yes").mean() * 100)  # % de clients ayant répondu "Yes"
).reset_index()

In [7]:
display(kpi_results)

,Sales Channel,Total_Customers,Avg_CLV,Total_Premium,Conversion_Rate
0,Agent,3477,7957.71,325722,19.15
1,Branch,2567,8119.71,239519,11.45
2,Call Center,1765,8100.09,163231,10.88
3,Web,1325,7779.79,122993,11.77


In [8]:
# Calculs supplémentaires : CPL, CPA, CRR, CCR, ROI
kpi_results["CPL"] = total_marketing_cost / kpi_results["Total_Customers"]
kpi_results["CPA"] = total_marketing_cost / (kpi_results["Conversion_Rate"] / 100 * kpi_results["Total_Customers"])
kpi_results["CRR"] = kpi_results["Conversion_Rate"]  # On suppose que les clients ayant répondu "Yes" sont retenus
kpi_results["CCR"] = 100 - kpi_results["CRR"]  # Attrition = 100 - Rétention
kpi_results["ROI"] = (kpi_results["Total_Premium"] - total_marketing_cost) / total_marketing_cost * 100

In [ ]:
display(kpi_results)

,Sales Channel,Total_Customers,Avg_CLV,Total_Premium,Conversion_Rate,CPL,CPA,CRR,CCR,ROI
0,Agent,3477,7957.71,325722,19.15,28.76,150.15,19.15,80.85,225.72
1,Branch,2567,8119.71,239519,11.45,38.96,340.14,11.45,88.55,139.52
2,Call Center,1765,8100.09,163231,10.88,56.66,520.83,10.88,89.12,63.23
3,Web,1325,7779.79,122993,11.77,75.47,641.03,11.77,88.23,22.99
